# DoorLoop CX Data - Customer Segmentation & Profile Analysis

This notebook builds comprehensive customer profiles by joining user and event data, then segments customers across multiple dimensions.

**Author**: CX Analyst
**Date**: 2025
**Purpose**: Customer Segmentation & Profile Building

In [414]:
# Import libraries
import sys
sys.path.append('../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta

# Custom modules
from utils.db_connector import DataConnector
from utils.data_processor import CXDataProcessor

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("Libraries loaded successfully!")

Libraries loaded successfully!


## 1. Load Data

In [415]:
# Load data from Snowflake or local CSV
with DataConnector() as db:
    users_df = db.load_users()
    events_df = db.load_events()

print(f"Users: {len(users_df):,} records")
print(f"Events: {len(events_df):,} records")

# Display sample
print("\nUsers Sample:")
display(users_df.head())

print("\nEvents Sample:")
display(events_df.head())

Users: 10,000 records
Events: 228,731 records

Users Sample:


,user_id,signup_date,plan_type,portfolio_size,annual_revenue,is_active,nps_score,support_tickets_last_90d,success_manager_assigned,csm_id,renewal_due_date
0,1,2024-04-08,starter,1,143.89,True,6.0,0,0,NaN,2026-04-08
1,2,2025-03-22,starter,40,177.41,False,-53.0,3,1,CSM009,2025-09-12
2,3,2024-08-14,starter,10,336.76,True,37.0,0,0,NaN,2026-08-14
3,4,2024-02-09,starter,30,450.13,True,51.0,0,0,NaN,2026-02-09
4,5,2025-06-18,pro,80,2172.85,True,24.0,1,1,CSM006,2026-06-18



Events Sample:


,event_id,user_id,event_ts,event_type,event_value_num,event_value_txt
0,1,1,2024-04-09,login,4.45,NaN
1,2,1,2024-05-10,login,15.59,NaN
2,3,1,2024-05-20,tenant_added,NaN,NaN
3,4,1,2024-05-29,login,34.86,NaN
4,5,1,2024-06-24,maintenance_request_created,NaN,electric


## 2. Create Joined Table

In [416]:
# Aggregate event-level metrics per user
event_metrics = events_df.groupby('user_id').agg({
    'event_ts': ['min', 'max', 'count'],
    'event_type': lambda x: x.value_counts().to_dict()
}).reset_index()

# Flatten column names
event_metrics.columns = ['user_id', 'first_event_date', 'last_event_date', 'total_events', 'event_type_breakdown']

# Calculate days since last activity
event_metrics['last_event_date'] = pd.to_datetime(event_metrics['last_event_date'])
event_metrics['days_since_last_activity'] = (datetime.now() - event_metrics['last_event_date']).dt.days

# Extract individual event type counts
event_type_counts = events_df.groupby(['user_id', 'event_type']).size().unstack(fill_value=0).reset_index()
event_type_counts.columns = ['user_id'] + [f'{col}_count' for col in event_type_counts.columns[1:]]

# Merge event metrics
event_metrics = event_metrics.merge(event_type_counts, on='user_id', how='left')

# Join users with event metrics
joined_df = users_df.merge(event_metrics, on='user_id', how='left')

# Fill NAs for users with no events
joined_df['total_events'] = joined_df['total_events'].fillna(0)
joined_df['days_since_last_activity'] = joined_df['days_since_last_activity'].fillna(999)

print(f"Joined table created: {joined_df.shape}")
print(f"Columns: {len(joined_df.columns)}")
print(f"\nColumn names: {list(joined_df.columns)}")
display(joined_df.head())

Joined table created: (10000, 27)
Columns: 27

Column names: ['user_id', 'signup_date', 'plan_type', 'portfolio_size', 'annual_revenue', 'is_active', 'nps_score', 'support_tickets_last_90d', 'success_manager_assigned', 'csm_id', 'renewal_due_date', 'first_event_date', 'last_event_date', 'total_events', 'event_type_breakdown', 'days_since_last_activity', 'feature_adopted_count', 'lease_signed_count', 'login_count', 'maintenance_request_created_count', 'nps_response_submitted_count', 'rent_payment_received_count', 'report_generated_count', 'subscription_cancelled_count', 'support_ticket_created_count', 'tenant_added_count', 'training_attended_count']


,user_id,signup_date,plan_type,portfolio_size,annual_revenue,is_active,nps_score,support_tickets_last_90d,success_manager_assigned,csm_id,renewal_due_date,first_event_date,last_event_date,total_events,event_type_breakdown,days_since_last_activity,feature_adopted_count,lease_signed_count,login_count,maintenance_request_created_count,nps_response_submitted_count,rent_payment_received_count,report_generated_count,subscription_cancelled_count,support_ticket_created_count,tenant_added_count,training_attended_count
0,1,2024-04-08,starter,1,143.89,True,6.0,0,0,NaN,2026-04-08,2024-04-09,2026-03-19,16,"{'login': 7, 'maintenance_request_created': 5,...",-139,0,0,7,5,1,0,1,0,0,2,0
1,2,2025-03-22,starter,40,177.41,False,-53.0,3,1,CSM009,2025-09-12,2025-03-23,2025-08-22,25,"{'maintenance_request_created': 5, 'login': 4,...",70,0,2,4,5,1,0,4,1,3,4,1
2,3,2024-08-14,starter,10,336.76,True,37.0,0,0,NaN,2026-08-14,2024-08-15,2026-07-12,15,"{'maintenance_request_created': 5, 'login': 3,...",-254,1,2,3,5,1,1,1,0,0,1,0
3,4,2024-02-09,starter,30,450.13,True,51.0,0,0,NaN,2026-02-09,2024-02-09,2026-01-11,21,"{'login': 5, 'lease_signed': 4, 'report_genera...",-72,1,4,5,3,1,2,3,0,0,2,0
4,5,2025-06-18,pro,80,2172.85,True,24.0,1,1,CSM006,2026-06-18,2025-06-18,2026-05-28,29,"{'report_generated': 8, 'lease_signed': 7, 'lo...",-209,0,7,5,1,1,0,8,0,1,5,1


In [417]:
joined_df.('joined_df.csv')

SyntaxError: invalid syntax (3131842840.py, line 1)

## 3. Portfolio Size Groups

In [ ]:
# Create portfolio size groups (4 ranges: 1, 2-5, 5-10, 10+)
joined_df['portfolio_group'] = pd.cut(
    joined_df['portfolio_size'],
    bins=[0, 1, 5, 10, float('inf')],
    labels=['1 unit', '2-5 units', '6-10 units', '10+ units'],
    include_lowest=True
)

# Check portfolio distribution
print("=== Portfolio Size Distribution ===")
print(joined_df['portfolio_group'].value_counts().sort_index())
print(f"\nPlan Type Distribution:")
print(joined_df['plan_type'].value_counts())

=== Portfolio Size Distribution ===
portfolio_group
1 unit         913
2-5 units     3355
6-10 units     883
10+ units     4849
Name: count, dtype: int64

Plan Type Distribution:
plan_type
starter    5207
pro        3286
premium    1507
Name: count, dtype: int64


In [ ]:
# Check for null or <1 values in portfolio_size
null_count = joined_df['portfolio_size'].isnull().sum()
less_than_one = (joined_df['portfolio_size'] < 1).sum()
zero_values = (joined_df['portfolio_size'] == 0).sum()

print("="*80)
print("PORTFOLIO SIZE DATA QUALITY CHECK")
print("="*80)
print(f"Total records: {len(joined_df):,}")
print(f"Null values: {null_count:,}")
print(f"Values < 1: {less_than_one:,}")
print(f"Values = 0: {zero_values:,}")
print(f"\nPortfolio size range: {joined_df['portfolio_size'].min()} to {joined_df['portfolio_size'].max()}")

PORTFOLIO SIZE DATA QUALITY CHECK
Total records: 10,000
Null values: 0
Values < 1: 199
Values = 0: 199

Portfolio size range: 0 to 100


In [ ]:
# Portfolio Size → Plan Type is our primary segmentation
# This view helps us understand customer behavior by scale first, then by plan tier
print(f"\n{'='*80}")
print(f"Primary Segmentation: Portfolio Size → Plan Type")
print(f"{'='*80}")


Primary Segmentation: Portfolio Size → Plan Type


## 4. Segmentation: Portfolio Size → Plan Type

In [ ]:
# Create segment identifier: Portfolio Group → Plan Type
joined_df['segment_v2'] = joined_df['portfolio_group'].astype(str) + ' → ' + joined_df['plan_type'].astype(str)

# Calculate total for percentage calculations
total_users = len(joined_df)

# List all segments with counts and percentages
segments_v2_dict = {}

print(f"\n{'='*80}")
print(f"SEGMENTATION OVERVIEW: Portfolio Size → Plan Type")
print(f"{'='*80}")
print(f"Total Users: {total_users:,}\n")

for segment_name in sorted(joined_df['segment_v2'].unique()):
    segment_users = joined_df[joined_df['segment_v2'] == segment_name].copy()
    segments_v2_dict[segment_name] = segment_users
    
    segment_count = len(segment_users)
    segment_pct = (segment_count / total_users) * 100
    
    print(f"{segment_name:40} {segment_count:5,} users ({segment_pct:5.1f}%)")

print(f"\n{'='*80}")
print(f"Total Segments: {len(segments_v2_dict)}")
print(f"{'='*80}")


SEGMENTATION OVERVIEW: Portfolio Size → Plan Type
Total Users: 10,000

1 unit → premium                           144 users (  1.4%)
1 unit → pro                               299 users (  3.0%)
1 unit → starter                           470 users (  4.7%)
10+ units → premium                        724 users (  7.2%)
10+ units → pro                          1,629 users ( 16.3%)
10+ units → starter                      2,496 users ( 25.0%)
2-5 units → premium                        509 users (  5.1%)
2-5 units → pro                          1,080 users ( 10.8%)
2-5 units → starter                      1,766 users ( 17.7%)
6-10 units → premium                       130 users (  1.3%)
6-10 units → pro                           278 users (  2.8%)
6-10 units → starter                       475 users (  4.8%)

Total Segments: 12


## 5. Aggregate Statistics for All Segments

In [ ]:
# Create comprehensive aggregate statistics for Portfolio Group → Plan Type segments

# First, identify all event type count columns
event_count_cols = [col for col in joined_df.columns if col.endswith('_count')]

# Build aggregation dictionary
agg_dict = {
    'user_id': 'count',
    'annual_revenue': ['sum', 'mean', 'median', 'min', 'max'],
    'portfolio_size': ['mean', 'median', 'min', 'max'],
    'nps_score': ['mean', 'median', 'std'],
    'support_tickets_last_90d': ['sum', 'mean', 'median', 'max'],
    'total_events': ['sum', 'mean', 'median'],
    'days_since_last_activity': ['mean', 'median', 'min', 'max'],
    'is_active': ['sum', lambda x: (x.sum() / len(x) * 100)]
}

# Add all event type count columns with sum aggregation
for col in event_count_cols:
    agg_dict[col] = 'sum'

# Create segment statistics
segment_stats = joined_df.groupby(['portfolio_group', 'plan_type']).agg(agg_dict).round(2)

# Flatten multi-level columns
segment_stats.columns = ['_'.join(col).strip() for col in segment_stats.columns.values]

# Rename for clarity
rename_dict = {
    'user_id_count': 'customer_count',
    'annual_revenue_sum': 'total_arr',
    'annual_revenue_mean': 'avg_arr',
    'annual_revenue_median': 'median_arr',
    'annual_revenue_min': 'min_arr',
    'annual_revenue_max': 'max_arr',
    'portfolio_size_mean': 'avg_portfolio',
    'portfolio_size_median': 'median_portfolio',
    'portfolio_size_min': 'min_portfolio',
    'portfolio_size_max': 'max_portfolio',
    'nps_score_mean': 'avg_nps',
    'nps_score_median': 'median_nps',
    'nps_score_std': 'std_nps',
    'support_tickets_last_90d_sum': 'total_tickets',
    'support_tickets_last_90d_mean': 'avg_tickets',
    'support_tickets_last_90d_median': 'median_tickets',
    'support_tickets_last_90d_max': 'max_tickets',
    'total_events_sum': 'total_events_all',
    'total_events_mean': 'avg_events_per_user',
    'total_events_median': 'median_events_per_user',
    'days_since_last_activity_mean': 'avg_days_since_activity',
    'days_since_last_activity_median': 'median_days_since_activity',
    'days_since_last_activity_min': 'min_days_since_activity',
    'days_since_last_activity_max': 'max_days_since_activity',
    'is_active_sum': 'active_customers'
}

# Rename event count columns (remove _sum suffix added by groupby)
for col in event_count_cols:
    rename_dict[f'{col}_sum'] = f'{col}_total'

segment_stats = segment_stats.rename(columns=rename_dict)

# Rename the lambda column properly
segment_stats.columns = [col.replace('is_active_<lambda_0>', 'active_rate_pct') if 'lambda' in col else col for col in segment_stats.columns]

print("="*100)
print("SEGMENT STATISTICS: Portfolio Group → Plan Type")
print("="*100)
print(f"\nEvent type columns included: {len(event_count_cols)}")
print(f"Event types: {', '.join([col.replace('_count', '') for col in event_count_cols])}")
print()
display(segment_stats)

# Save to variable for easy access
segment_stats_summary = segment_stats.copy()

SEGMENT STATISTICS: Portfolio Group → Plan Type

Event type columns included: 11
Event types: feature_adopted, lease_signed, login, maintenance_request_created, nps_response_submitted, rent_payment_received, report_generated, subscription_cancelled, support_ticket_created, tenant_added, training_attended



/var/folders/dt/ygs29g2d69z6j_kp3w6648340000gn/T/ipykernel_86856/3296442609.py:23: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



customer_count    total_arr   avg_arr  median_arr  min_arr   max_arr  avg_portfolio  median_portfolio  min_portfolio  max_portfolio  avg_nps  median_nps  std_nps  total_tickets  avg_tickets  median_tickets  max_tickets  total_events_all  avg_events_per_user  median_events_per_user  avg_days_since_activity  median_days_since_activity  min_days_since_activity  max_days_since_activity  active_customers  active_rate_pct  feature_adopted_count_total  lease_signed_count_total  login_count_total  maintenance_request_created_count_total  nps_response_submitted_count_total  rent_payment_received_count_total  report_generated_count_total  subscription_cancelled_count_total  support_ticket_created_count_total  tenant_added_count_total  training_attended_count_total
portfolio_group plan_type                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
1 unit          premium               144   1953914.30  13568.85    12852.09  3087.16  24109.40           0.83               1.0              0              1    23.86        30.5    32.68            212         1.47             0.0            8              3795                26.35                    26.0                   -59.70                       -79.5                     -303                      508               121            84.03                           73                       259                795                                      576                                 132                                137                          1131                                  23                                 212                       367                             90
                pro                   299    547380.66   1830.70     1842.18   604.60   3012.25           0.79               1.0              0              1    19.61        24.0    34.07            241         0.81             0.0            8              6006                20.09                    20.0                   -27.63                       -54.0                     -321                      607               232            77.59                          137                       488               1581                                      805                                 274                                303                          1283                                  67                                 241                       707                            120
                starter               470    169239.03    360.08      356.54   120.93    601.94           0.76               1.0              0              1    17.61        19.0    34.29            327         0.70             0.0            8              7786                16.57                    16.0                    27.60                         2.0                     -318                      660               296            62.98                          162                       674               2376                                      980                                 426                                330                          1295                                 174                                 327                       967                             75
2-5 units       premium               509   6663474.96  13091.31   

In [ ]:
# Create visualizations of key segment metrics

# 1. Customer Count by Segment
fig = px.bar(segment_stats.reset_index(), 
             x='portfolio_group', 
             y='customer_count',
             color='plan_type',
             title='Customer Count by Portfolio Group and Plan Type',
             barmode='group',
             labels={'customer_count': 'Number of Customers', 'portfolio_group': 'Portfolio Group'})
fig.show()

# 2. Total ARR by Segment
fig = px.bar(segment_stats.reset_index(), 
             x='portfolio_group', 
             y='total_arr',
             color='plan_type',
             title='Total ARR by Portfolio Group and Plan Type',
             barmode='group',
             labels={'total_arr': 'Total Annual Revenue ($)', 'portfolio_group': 'Portfolio Group'})
fig.show()

# 3. Average NPS by Segment
fig = px.bar(segment_stats.reset_index(), 
             x='portfolio_group', 
             y='avg_nps',
             color='plan_type',
             title='Average NPS by Portfolio Group and Plan Type',
             barmode='group',
             labels={'avg_nps': 'Average NPS Score', 'portfolio_group': 'Portfolio Group'})
fig.show()

# 4. Active Rate by Segment
fig = px.bar(segment_stats.reset_index(), 
             x='portfolio_group', 
             y='active_rate_pct',
             color='plan_type',
             title='Active Rate (%) by Portfolio Group and Plan Type',
             barmode='group',
             labels={'active_rate_pct': 'Active Rate (%)', 'portfolio_group': 'Portfolio Group'})
fig.show()

## 6. User Flow Analysis: Sankey Diagram

In [ ]:
# First, check the distribution of total_events to inform engagement tiers
print("="*80)
print("EVENT DISTRIBUTION ANALYSIS")
print("="*80)
print(f"Total events stats:")
print(joined_df['total_events'].describe())
print(f"\nAverage events per user: {joined_df['total_events'].mean():.1f}")
print(f"Median events per user: {joined_df['total_events'].median():.1f}")

# Create engagement tiers based on more granular ranges
# Using <15 = Low, 15-30 = Medium, 30+ = High
joined_df['engagement_level'] = pd.cut(
    joined_df['total_events'],
    bins=[-1, 15, 30, float('inf')],
    labels=['Low (<15 events)', 'Medium (15-30 events)', 'High (30+ events)']
)

# Create active status label
joined_df['active_status'] = joined_df['is_active'].map({1: 'Active', 0: 'Inactive'})

# Check distribution
print("\n" + "="*80)
print("ENGAGEMENT LEVEL DISTRIBUTION")
print("="*80)
print(joined_df['engagement_level'].value_counts().sort_index())
print(f"\nActive Status Distribution:")
print(joined_df['active_status'].value_counts())

# Show average events per engagement level to validate ranges
print("\n" + "="*80)
print("VALIDATION: Average events per engagement level")
print("="*80)
engagement_validation = joined_df.groupby('engagement_level')['total_events'].agg(['count', 'mean', 'min', 'max']).round(1)
display(engagement_validation)

EVENT DISTRIBUTION ANALYSIS
Total events stats:
count    10000.000000
mean        22.873100
std          6.617279
min          5.000000
25%         18.000000
50%         22.000000
75%         27.000000
max         51.000000
Name: total_events, dtype: float64

Average events per user: 22.9
Median events per user: 22.0

ENGAGEMENT LEVEL DISTRIBUTION
engagement_level
Low (<15 events)         1329
Medium (15-30 events)    7334
High (30+ events)        1337
Name: count, dtype: int64

Active Status Distribution:
Series([], Name: count, dtype: int64)

VALIDATION: Average events per engagement level


/var/folders/dt/ygs29g2d69z6j_kp3w6648340000gn/T/ipykernel_86856/2738385316.py:33: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



,count,mean,min,max
engagement_level,,,,
Low (<15 events),1329,13.9,5,15
Medium (15-30 events),7334,22.3,16,30
High (30+ events),1337,34.8,31,51


In [ ]:
# Build Sankey flow data
# Flow: Portfolio Size → Plan Type → Engagement Level → Active Status

# Ensure required columns exist before grouping (always recreate to ensure consistency)
joined_df['portfolio_group'] = pd.cut(
    joined_df['portfolio_size'],
    bins=[0, 1, 5, 10, float('inf')],
    labels=['1 unit', '2-5 units', '6-10 units', '10+ units'],
    include_lowest=True
)

joined_df['engagement_level'] = pd.cut(
    joined_df['total_events'],
    bins=[-1, 15, 30, float('inf')],
    labels=['Low (<15 events)', 'Medium (15-30 events)', 'High (30+ events)']
)

joined_df['active_status'] = joined_df['is_active'].map({True: 'Active', False: 'Inactive', 1: 'Active', 0: 'Inactive'})
joined_df['active_status'] = joined_df['active_status'].fillna('Unknown')

# Debug: Check column status
print("DEBUG: Checking columns before groupby...")
print(f"  portfolio_group exists: {'portfolio_group' in joined_df.columns}")
print(f"  engagement_level exists: {'engagement_level' in joined_df.columns}")
print(f"  active_status exists: {'active_status' in joined_df.columns}")
print(f"  plan_type exists: {'plan_type' in joined_df.columns}")

if 'portfolio_group' in joined_df.columns:
    print(f"  portfolio_group null count: {joined_df['portfolio_group'].isnull().sum()}")
    print(f"  portfolio_group unique values: {joined_df['portfolio_group'].unique()}")
    
if 'engagement_level' in joined_df.columns:
    print(f"  engagement_level null count: {joined_df['engagement_level'].isnull().sum()}")
    print(f"  engagement_level unique values: {joined_df['engagement_level'].unique()}")
    
if 'active_status' in joined_df.columns:
    print(f"  active_status null count: {joined_df['active_status'].isnull().sum()}")
    print(f"  active_status unique values: {joined_df['active_status'].unique()}")

# Filter out rows with NaN in grouping columns and convert categoricals to strings for groupby
df_for_groupby = joined_df.dropna(subset=['portfolio_group', 'plan_type', 'engagement_level', 'active_status']).copy()

# Convert categorical columns to strings to avoid groupby issues
for col in ['portfolio_group', 'engagement_level', 'active_status']:
    if col in df_for_groupby.columns:
        df_for_groupby[col] = df_for_groupby[col].astype(str)

print(f"  Rows after dropping NaN: {len(df_for_groupby)} (from {len(joined_df)})")

# Create flow aggregations
flow_data = df_for_groupby.groupby([
    'portfolio_group', 
    'plan_type', 
    'engagement_level', 
    'active_status'
], observed=True).agg({
    'user_id': 'count',
    'annual_revenue': 'sum'
}).reset_index()

flow_data.columns = ['portfolio_group', 'plan_type', 'engagement_level', 'active_status', 'count', 'total_arr']

# Calculate percentages
total_users = flow_data['count'].sum()
# Calculate percentages safely
if len(flow_data) > 0:
    total_users = flow_data['count'].sum()
    if total_users > 0:
        flow_data['percentage'] = (flow_data['count'] / total_users * 100).round(2)
    else:
        flow_data['percentage'] = 0
else:
    flow_data['percentage'] = 0

print("="*80)
print("FLOW DATA SUMMARY")
print("="*80)
print(f"Total flow paths: {len(flow_data)}")
print(f"Total users: {total_users:,}")
print(f"Total ARR: ${flow_data['total_arr'].sum():,.0f}")
print(f"\nTop 10 paths by customer count:")
display(flow_data.nlargest(10, 'count')[['portfolio_group', 'plan_type', 'engagement_level', 'active_status', 'count', 'percentage', 'total_arr']])

DEBUG: Checking columns before groupby...
  portfolio_group exists: True
  engagement_level exists: True
  active_status exists: True
  plan_type exists: True
  portfolio_group null count: 0
  portfolio_group unique values: ['1 unit', '10+ units', '6-10 units', '2-5 units']
Categories (4, object): ['1 unit' < '2-5 units' < '6-10 units' < '10+ units']
  engagement_level null count: 0
  engagement_level unique values: ['Medium (15-30 events)', 'Low (<15 events)', 'High (30+ events)']
Categories (3, object): ['Low (<15 events)' < 'Medium (15-30 events)' < 'High (30+ events)']
  active_status null count: 0
  active_status unique values: ['Active' 'Inactive']
  Rows after dropping NaN: 10000 (from 10000)
FLOW DATA SUMMARY
Total flow paths: 62
Total users: 10,000
Total ARR: $30,692,996

Top 10 paths by customer count:


,portfolio_group,plan_type,engagement_level,active_status,count,percentage,total_arr
29,10+ units,starter,Medium (15-30 events),Active,1370,13.70,587061.18
23,10+ units,pro,Medium (15-30 events),Active,942,9.42,1976751.06
45,2-5 units,starter,Medium (15-30 events),Active,793,7.93,293856.11
39,2-5 units,pro,Medium (15-30 events),Active,732,7.32,1331559.50
30,10+ units,starter,Medium (15-30 events),Inactive,728,7.28,314551.58
46,2-5 units,starter,Medium (15-30 events),Inactive,409,4.09,150418.31
15,10+ units,premium,High (30+ events),Active,397,3.97,6408801.53
43,2-5 units,starter,Low (<15 events),Active,353,3.53,130498.79
19,10+ units,pro,High (30+ events),Active,296,2.96,640872.40
24,10+ units,pro,Medium (15-30 events),Inactive,275,2.75,575630.39


In [ ]:
# Create Combined Sankey diagram
# Flow: Size → Plan → Events → Tickets → Active Status → ARR Group

# Ensure required columns exist
if 'portfolio_group' not in joined_df.columns:
    joined_df['portfolio_group'] = pd.cut(
        joined_df['portfolio_size'],
        bins=[0, 1, 5, 10, float('inf')],
        labels=['1 unit', '2-5 units', '6-10 units', '10+ units'],
        include_lowest=True
    )

if 'engagement_level' not in joined_df.columns:
    joined_df['engagement_level'] = pd.cut(
        joined_df['total_events'],
        bins=[-1, 15, 30, float('inf')],
        labels=['Low (<15 events)', 'Medium (15-30 events)', 'High (30+ events)']
    )

if 'active_status' not in joined_df.columns:
    joined_df['active_status'] = joined_df['is_active'].map({True: 'Active', False: 'Inactive', 1: 'Active', 0: 'Inactive'})
    joined_df['active_status'] = joined_df['active_status'].fillna('Unknown')

# Ensure arr_group exists
if 'arr_group' not in joined_df.columns:
    joined_df['arr_group'] = pd.cut(
        joined_df['annual_revenue'],
        bins=[0, 5000, 15000, float('inf')],
        labels=['Low ARR (<$5k)', 'Mid ARR ($5k-$15k)', 'High ARR ($15k+)']
    )

# Create arr_group with inactive handling
df_with_arr = joined_df.copy()
df_with_arr['arr_group'] = df_with_arr['arr_group'].astype(str)
df_with_arr.loc[df_with_arr['is_active'] == False, 'arr_group'] = 'Inactive (No ARR)'
df_with_arr.loc[df_with_arr['is_active'] == 0, 'arr_group'] = 'Inactive (No ARR)'

# Create ticket range groups
joined_df['ticket_range'] = pd.cut(
    joined_df['support_tickets_last_90d'],
    bins=[-1, 0, 1, 4, 10, float('inf')],
    labels=['0 tickets', '1 ticket', '2-4 tickets', '5-10 tickets', '11+ tickets'],
    include_lowest=True
)

# Prepare data - convert categoricals to strings and drop rows with missing values
df_combined = joined_df.dropna(subset=['portfolio_group', 'plan_type', 'arr_group', 'engagement_level', 'ticket_range', 'active_status']).copy()
for col in ['portfolio_group', 'plan_type', 'engagement_level', 'ticket_range', 'active_status']:
    df_combined[col] = df_combined[col].astype(str)
df_combined['arr_group'] = df_with_arr.loc[df_combined.index, 'arr_group'].astype(str)

# Build nodes in order: Size → Plan → Events → Tickets → Active Status → ARR Group
node_labels = []

# Level 1: Portfolio Groups (Size)
portfolio_nodes = sorted([str(x) for x in df_combined['portfolio_group'].unique()])
node_labels.extend(portfolio_nodes)

# Level 2: Plan Types
plan_nodes = sorted([str(x) for x in df_combined['plan_type'].unique()])
node_labels.extend(plan_nodes)

# Level 3: Engagement Levels (Events)
engagement_nodes = sorted([str(x) for x in df_combined['engagement_level'].unique()])
node_labels.extend(engagement_nodes)

# Level 4: Ticket Ranges
ticket_range_nodes = sorted([str(x) for x in df_combined['ticket_range'].unique()])
node_labels.extend(ticket_range_nodes)

# Level 5: Active Status
status_nodes = sorted([str(x) for x in df_combined['active_status'].unique()])
node_labels.extend(status_nodes)

# Level 6: ARR Groups
arr_nodes = sorted([str(x) for x in df_combined['arr_group'].unique()])
node_labels.extend(arr_nodes)

# Create node index mapping (removes duplicates automatically since dict keys are unique)
node_dict = {str(label): idx for idx, label in enumerate(node_labels)}

# Build links
sources = []
targets = []
values = []
link_labels = []

# Flow 1: Portfolio Group (Size) → Plan Type
flow1 = df_combined.groupby(['portfolio_group', 'plan_type'], observed=True).agg({
    'user_id': 'count',
    'annual_revenue': 'sum'
}).reset_index()

for _, row in flow1.iterrows():
    sources.append(node_dict[str(row['portfolio_group'])])
    targets.append(node_dict[str(row['plan_type'])])
    values.append(row['user_id'])
    link_labels.append(f"{row['user_id']} users, ${row['annual_revenue']:,.0f} ARR")

# Flow 2: Plan Type → Engagement Level (Events)
flow2 = df_combined.groupby(['plan_type', 'engagement_level'], observed=True).agg({
    'user_id': 'count',
    'annual_revenue': 'sum'
}).reset_index()

for _, row in flow2.iterrows():
    sources.append(node_dict[str(row['plan_type'])])
    targets.append(node_dict[str(row['engagement_level'])])
    values.append(row['user_id'])
    link_labels.append(f"{row['user_id']} users, ${row['annual_revenue']:,.0f} ARR")

# Flow 3: Engagement Level → Ticket Range
flow3 = df_combined.groupby(['engagement_level', 'ticket_range'], observed=True).agg({
    'user_id': 'count',
    'annual_revenue': 'sum',
    'support_tickets_last_90d': 'sum'
}).reset_index()

for _, row in flow3.iterrows():
    sources.append(node_dict[str(row['engagement_level'])])
    targets.append(node_dict[str(row['ticket_range'])])
    values.append(row['user_id'])
    link_labels.append(f"{row['user_id']} users, ${row['annual_revenue']:,.0f} ARR, {row['support_tickets_last_90d']:.0f} tickets")

# Flow 4: Ticket Range → Active Status
flow4 = df_combined.groupby(['ticket_range', 'active_status'], observed=True).agg({
    'user_id': 'count',
    'annual_revenue': 'sum',
    'support_tickets_last_90d': 'sum'
}).reset_index()

for _, row in flow4.iterrows():
    sources.append(node_dict[str(row['ticket_range'])])
    targets.append(node_dict[str(row['active_status'])])
    values.append(row['user_id'])
    link_labels.append(f"{row['user_id']} users, ${row['annual_revenue']:,.0f} ARR, {row['support_tickets_last_90d']:.0f} tickets")

# Flow 5: Active Status → ARR Group
flow5 = df_combined.groupby(['active_status', 'arr_group'], observed=True).agg({
    'user_id': 'count',
    'annual_revenue': 'sum'
}).reset_index()

for _, row in flow5.iterrows():
    sources.append(node_dict[str(row['active_status'])])
    targets.append(node_dict[str(row['arr_group'])])
    values.append(row['user_id'])
    link_labels.append(f"{row['user_id']} users, ${row['annual_revenue']:,.0f} ARR")

# Create Sankey diagram
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=node_labels,
        color=['lightblue' if 'unit' in str(n) else 
               'lightgreen' if str(n) in ['starter', 'pro', 'enterprise', 'premium'] else
               'gold' if 'ARR' in str(n) else
               'lightyellow' if 'events' in str(n).lower() or '<15' in str(n) or '15-30' in str(n) or '30+' in str(n) else
               'orange' if 'ticket' in str(n).lower() else
               'lightcoral' if str(n) == 'Inactive' else
               'lightseagreen' if str(n) == 'Active' else
               'salmon'
               for n in node_labels]
    ),
    link=dict(
        source=sources,
        target=targets,
        value=values,
        label=link_labels
    )
)])

fig.update_layout(
    title="User Flow: Portfolio Size → Plan Type → Engagement (Events) → Ticket Range → Active Status → ARR Group",
    font=dict(size=10),
    height=900
)

fig.show()

print("\n" + "="*80)
print("COMBINED SANKEY DIAGRAM LEGEND")
print("="*80)
print("Flow shows how customers progress through:")
print("  1. Portfolio Size (blue)")
print("  2. Plan Type (green)")
print("  3. Engagement Level / Events (yellow)")
print("  4. Ticket Range (orange)")
print("  5. Active Status (teal=Active, coral=Inactive)")
print("  6. ARR Group (gold)")
print("\nHover over flows to see customer count, total ARR, and ticket counts")


COMBINED SANKEY DIAGRAM LEGEND
Flow shows how customers progress through:
  1. Portfolio Size (blue)
  2. Plan Type (green)
  3. ARR Group (gold)
  4. Engagement Level / Events (yellow)
  5. Ticket Range (orange)
  6. Active Status (teal=Active, coral=Inactive)

Hover over flows to see customer count, total ARR, and ticket counts


In [ ]:
# NOTE: This Sankey diagram has been combined with the previous one
# See cell above for the combined flow: Size → Plan → ARR Group → Events → Tickets → Active Status

# Ensure required columns exist
if 'engagement_level' not in joined_df.columns:
    joined_df['engagement_level'] = pd.cut(
        joined_df['total_events'],
        bins=[-1, 15, 30, float('inf')],
        labels=['Low (<15 events)', 'Medium (15-30 events)', 'High (30+ events)']
    )

if 'active_status' not in joined_df.columns:
    joined_df['active_status'] = joined_df['is_active'].map({True: 'Active', False: 'Inactive', 1: 'Active', 0: 'Inactive'})
    joined_df['active_status'] = joined_df['active_status'].fillna('Unknown')

# Create ticket range groups based on support_tickets_last_90d
# Ranges: 0, 1, 2-4, 5-10, 11+
joined_df['ticket_range'] = pd.cut(
    joined_df['support_tickets_last_90d'],
    bins=[-1, 0, 1, 4, 10, float('inf')],
    labels=['0 tickets', '1 ticket', '2-4 tickets', '5-10 tickets', '11+ tickets'],
    include_lowest=True
)

# Prepare data - convert categoricals to strings
df_support = joined_df.dropna(subset=['engagement_level', 'ticket_range', 'active_status', 'support_tickets_last_90d']).copy()
for col in ['engagement_level', 'ticket_range', 'active_status']:
    df_support[col] = df_support[col].astype(str)

# Build nodes
node_labels = []

# Level 1: Engagement Levels
engagement_nodes = sorted([str(x) for x in df_support['engagement_level'].unique()])
node_labels.extend(engagement_nodes)

# Level 2: Ticket Ranges
ticket_range_nodes = sorted([str(x) for x in df_support['ticket_range'].unique()])
node_labels.extend(ticket_range_nodes)

# Level 3: Active Status
status_nodes = sorted([str(x) for x in df_support['active_status'].unique()])
node_labels.extend(status_nodes)

# Create node index mapping
node_dict = {str(label): idx for idx, label in enumerate(node_labels)}

# Build links
sources = []
targets = []
values = []
link_labels = []

# Flow 1: Engagement Level → Ticket Range
flow1 = df_support.groupby(['engagement_level', 'ticket_range'], observed=True).agg({
    'support_tickets_last_90d': 'sum',
    'user_id': 'count',
    'annual_revenue': 'sum'
}).reset_index()

for _, row in flow1.iterrows():
    sources.append(node_dict[str(row['engagement_level'])])
    targets.append(node_dict[str(row['ticket_range'])])
    values.append(row['support_tickets_last_90d'])
    link_labels.append(
        f"{row['support_tickets_last_90d']:.0f} tickets, "
        f"{row['user_id']} users, "
        f"${row['annual_revenue']:,.0f} ARR"
    )

# Flow 2: Ticket Range → Active Status
flow2 = df_support.groupby(['ticket_range', 'active_status'], observed=True).agg({
    'support_tickets_last_90d': 'sum',
    'user_id': 'count',
    'annual_revenue': 'sum'
}).reset_index()

for _, row in flow2.iterrows():
    sources.append(node_dict[str(row['ticket_range'])])
    targets.append(node_dict[str(row['active_status'])])
    values.append(row['support_tickets_last_90d'])
    link_labels.append(
        f"{row['support_tickets_last_90d']:.0f} tickets, "
        f"{row['user_id']} users, "
        f"${row['annual_revenue']:,.0f} ARR"
    )

# Create Sankey diagram
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=30,
        line=dict(color="black", width=0.5),
        label=node_labels,
        color=['lightyellow' if 'events' in str(n) else
               'orange' if 'ticket' in str(n) else
               'lightcoral' if str(n) == 'Inactive' else
               'lightseagreen'
               for n in node_labels]
    ),
    link=dict(
        source=sources,
        target=targets,
        value=values,
        label=link_labels,
        # Color links based on ticket volume intensity
        color=[f'rgba(255, 100, 100, {min(0.8, v/max(values) * 0.8 + 0.2)})' for v in values]
    )
)])

fig.update_layout(
    title="Support Ticket Flow: Engagement Level → Ticket Range → Active Status",
    font=dict(size=12),
    height=700,
    annotations=[
        dict(
            text=f"Total Tickets: {sum(values):,.0f}<br>"
                 f"Total Customers: {df_support['user_id'].nunique():,}",
            showarrow=False,
            xref="paper", yref="paper",
            x=0.5, y=-0.1,
            xanchor="center", yanchor="top",
            font=dict(size=11)
        )
    ]
)

fig.show()

print("\n" + "="*80)
print("SUPPORT TICKET FLOW ANALYSIS")
print("="*80)
print(f"Total support tickets (last 90 days): {sum(values):,.0f}")
print(f"Average tickets per customer: {sum(values) / df_support['user_id'].nunique():.2f}")
print(f"\nFlow shows:")
print("  • Engagement Level (yellow) → Ticket Range (orange) → Active Status (green=Active, coral=Inactive)")
print("  • Flow thickness = number of support tickets")
print("  • Hover to see ticket count, customer count, and ARR")
print("\n" + "="*80)
print("SUMMARY BY ENGAGEMENT LEVEL:")
print("="*80)
support_by_engagement = df_support.groupby('engagement_level')['support_tickets_last_90d'].agg(['sum', 'mean', 'count']).round(1)
support_by_engagement.columns = ['Total Tickets', 'Avg per Customer', 'Customer Count']
print(support_by_engagement)
print("\n" + "="*80)
print("SUMMARY BY TICKET RANGE:")
print("="*80)
support_by_range = df_support.groupby('ticket_range')['support_tickets_last_90d'].agg(['sum', 'mean', 'count']).round(1)
support_by_range.columns = ['Total Tickets', 'Avg per Customer', 'Customer Count']
support_by_range = support_by_range.sort_index()
print(support_by_range)
print("\n" + "="*80)
print("SUMMARY BY ACTIVE STATUS:")
print("="*80)
support_by_status = df_support.groupby('active_status')['support_tickets_last_90d'].agg(['sum', 'mean', 'count']).round(1)
support_by_status.columns = ['Total Tickets', 'Avg per Customer', 'Customer Count']
print(support_by_status)



SUPPORT TICKET FLOW ANALYSIS
Total support tickets (last 90 days): 24,682
Average tickets per customer: 2.47

Flow shows:
  • Engagement Level (yellow) → Ticket Range (orange) → Active Status (green=Active, coral=Inactive)
  • Flow thickness = number of support tickets
  • Hover to see ticket count, customer count, and ARR

SUMMARY BY ENGAGEMENT LEVEL:
                       Total Tickets  Avg per Customer  Customer Count
engagement_level                                                      
High (30+ events)               4150               3.1            1337
Low (<15 events)                 446               0.3            1329
Medium (15-30 events)           7745               1.1            7334

SUMMARY BY TICKET RANGE:
              Total Tickets  Avg per Customer  Customer Count
ticket_range                                                 
0 tickets                 0               0.0            4895
1 ticket               2237               1.0            2237
11+ tickets    

In [ ]:
# Summary table of complete flow paths with count, %, and ARR
# Ensure required columns exist before grouping
if 'portfolio_group' not in joined_df.columns:
    joined_df['portfolio_group'] = pd.cut(
        joined_df['portfolio_size'],
        bins=[0, 1, 5, 10, float('inf')],
        labels=['1 unit', '2-5 units', '6-10 units', '10+ units'],
        include_lowest=True
    )

if 'engagement_level' not in joined_df.columns:
    joined_df['engagement_level'] = pd.cut(
        joined_df['total_events'],
        bins=[-1, 15, 30, float('inf')],
        labels=['Low (<15 events)', 'Medium (15-30 events)', 'High (30+ events)']
    )

if 'active_status' not in joined_df.columns:
    joined_df['active_status'] = joined_df['is_active'].map({True: 'Active', False: 'Inactive', 1: 'Active', 0: 'Inactive'})
    # Fill any NaN values
    joined_df['active_status'] = joined_df['active_status'].fillna('Unknown')

complete_flow = joined_df.groupby([
    'portfolio_group', 
    'plan_type', 
    'engagement_level', 
    'active_status'
], observed=True).agg({
    'user_id': 'count',
    'annual_revenue': 'sum'
}).reset_index()

complete_flow.columns = ['Portfolio Size', 'Plan Type', 'Engagement', 'Status', 'Count', 'Total ARR']

# Check if dataframe is empty or has zero count
if len(complete_flow) == 0:
    print("Warning: No data found for complete flow paths. Check that all required columns are properly populated.")
    complete_flow['%'] = 0
    complete_flow['Avg ARR'] = 0
else:
    # Calculate percentage safely (avoid division by zero)
    total_count = complete_flow['Count'].sum()
    if total_count > 0:
        complete_flow['%'] = (complete_flow['Count'] / total_count * 100).round(2)
    else:
        complete_flow['%'] = 0
    
    # Calculate average ARR safely (avoid division by zero)
    complete_flow['Avg ARR'] = (complete_flow['Total ARR'] / complete_flow['Count'].replace(0, 1)).round(0)
    # Set Avg ARR to 0 where Count is 0
    complete_flow.loc[complete_flow['Count'] == 0, 'Avg ARR'] = 0

# Sort by count descending
complete_flow = complete_flow.sort_values('Count', ascending=False)

print("="*100)
print("COMPLETE FLOW PATHS: Portfolio Size → Plan Type → Engagement → Active Status")
print("="*100)
display(complete_flow)

# Key insights
print("\n" + "="*100)
print("KEY INSIGHTS")
print("="*100)

# Largest segments
top_3 = complete_flow.head(3)
print("\nTop 3 Customer Segments:")
for idx, row in top_3.iterrows():
    print(f"  • {row['Portfolio Size']} → {row['Plan Type']} → {row['Engagement']} → {row['Status']}")
    print(f"    {row['Count']} users ({row['%']}%), ${row['Total ARR']:,.0f} total ARR, ${row['Avg ARR']:,.0f} avg ARR")

# Highest ARR segments
top_arr = complete_flow.nlargest(3, 'Total ARR')
print("\nHighest ARR Segments:")
for idx, row in top_arr.iterrows():
    print(f"  • {row['Portfolio Size']} → {row['Plan Type']} → {row['Engagement']} → {row['Status']}")
    print(f"    {row['Count']} users ({row['%']}%), ${row['Total ARR']:,.0f} total ARR, ${row['Avg ARR']:,.0f} avg ARR")

# Inactive segments
inactive_segments = complete_flow[complete_flow['Status'] == 'Inactive']
print(f"\nInactive Customer Segments: {len(inactive_segments)} paths")
print(f"Total inactive users: {inactive_segments['Count'].sum()} ({(inactive_segments['Count'].sum()/complete_flow['Count'].sum()*100):.1f}%)")
print(f"ARR at risk: ${inactive_segments['Total ARR'].sum():,.0f}")

COMPLETE FLOW PATHS: Portfolio Size → Plan Type → Engagement → Active Status


,Portfolio Size,Plan Type,Engagement,Status,Count,Total ARR,%,Avg ARR
58,10+ units,starter,Medium (15-30 events),Active,1370,587061.18,13.70,429.0
52,10+ units,pro,Medium (15-30 events),Active,942,1976751.06,9.42,2098.0
27,2-5 units,starter,Medium (15-30 events),Active,793,293856.11,7.93,371.0
21,2-5 units,pro,Medium (15-30 events),Active,732,1331559.50,7.32,1819.0
59,10+ units,starter,Medium (15-30 events),Inactive,728,314551.58,7.28,432.0
...,...,...,...,...,...,...,...,...
0,1 unit,premium,Low (<15 events),Active,2,22329.40,0.02,11165.0
10,1 unit,pro,High (30+ events),Active,1,2440.90,0.01,2441.0
45,6-10 units,starter,High (30+ events),Inactive,1,412.52,0.01,413.0
30,2-5 units,starter,High (30+ events),Inactive,1,571.80,0.01,572.0



KEY INSIGHTS

Top 3 Customer Segments:
  • 10+ units → starter → Medium (15-30 events) → Active
    1370 users (13.7%), $587,061 total ARR, $429 avg ARR
  • 10+ units → pro → Medium (15-30 events) → Active
    942 users (9.42%), $1,976,751 total ARR, $2,098 avg ARR
  • 2-5 units → starter → Medium (15-30 events) → Active
    793 users (7.93%), $293,856 total ARR, $371 avg ARR

Highest ARR Segments:
  • 10+ units → premium → High (30+ events) → Active
    397 users (3.97%), $6,408,802 total ARR, $16,143 avg ARR
  • 2-5 units → premium → Medium (15-30 events) → Active
    267 users (2.67%), $3,625,812 total ARR, $13,580 avg ARR
  • 10+ units → premium → Medium (15-30 events) → Active
    155 users (1.55%), $2,416,356 total ARR, $15,589 avg ARR

Inactive Customer Segments: 31 paths
Total inactive users: 2959 (29.6%)
ARR at risk: $7,375,884


## 7. Event-Specific Sankey Diagrams

In [ ]:
# Identify all event type count columns (excluding login)
event_count_cols = [col for col in joined_df.columns if col.endswith('_count') and 'login' not in col.lower()]

print("="*80)
print("EVENT TYPE COLUMNS DETECTED (excluding login)")
print("="*80)
print(f"Total event types: {len(event_count_cols)}")
print(f"\nEvent types:")
for col in event_count_cols:
    event_name = col.replace('_count', '')
    total = joined_df[col].sum()
    users_with_event = (joined_df[col] > 0).sum()
    print(f"  • {event_name}: {total:,} total occurrences, {users_with_event} users have used it")

# Create event diversity metric - count how many different event types each user has used
joined_df['event_diversity'] = (joined_df[event_count_cols] > 0).sum(axis=1)

# Create event diversity groups: 0-4, 5-7, 7+
joined_df['event_diversity_group'] = pd.cut(
    joined_df['event_diversity'],
    bins=[-1, 4, 7, float('inf')],
    labels=['0-4 events', '5-7 events', '7+ events']
)

print(f"\n" + "="*80)
print("EVENT DIVERSITY DISTRIBUTION")
print("="*80)
print(joined_df['event_diversity_group'].value_counts().sort_index())
print(f"\nEvent diversity stats:")
print(joined_df['event_diversity'].describe())

# Create ARR groups for revenue categorization
joined_df['arr_group'] = pd.cut(
    joined_df['annual_revenue'],
    bins=[0, 5000, 15000, float('inf')],
    labels=['Low ARR (<$5k)', 'Mid ARR ($5k-$15k)', 'High ARR ($15k+)']
)

print(f"\n" + "="*80)
print("ARR GROUP DISTRIBUTION")
print("="*80)
print(joined_df['arr_group'].value_counts().sort_index())

EVENT TYPE COLUMNS DETECTED (excluding login)
Total event types: 10

Event types:
  • feature_adopted: 4,174 total occurrences, 4174 users have used it
  • lease_signed: 24,847 total occurrences, 8876 users have used it
  • maintenance_request_created: 37,288 total occurrences, 9495 users have used it
  • nps_response_submitted: 8,988 total occurrences, 8988 users have used it
  • rent_payment_received: 15,184 total occurrences, 6575 users have used it
  • report_generated: 36,479 total occurrences, 9370 users have used it
  • subscription_cancelled: 2,959 total occurrences, 2959 users have used it
  • support_ticket_created: 12,341 total occurrences, 5105 users have used it
  • tenant_added: 34,486 total occurrences, 9662 users have used it
  • training_attended: 3,228 total occurrences, 3228 users have used it

EVENT DIVERSITY DISTRIBUTION
event_diversity_group
0-4 events     443
5-7 events    6391
7+ events     3166
Name: count, dtype: int64

Event diversity stats:
count    10000.00

In [ ]:
# Function to create Sankey diagram showing event diversity
def create_event_diversity_sankey():
    """
    Create a Sankey diagram for: Portfolio → Plan → Event Diversity → Active Status → ARR Group
    Event diversity shows how many different event types users have adopted
    """
    df_clean = joined_df.copy()
    
    # Filter to only active users for the final ARR grouping
    df_with_arr = df_clean.copy()
    # Ensure arr_group exists, create if it doesn't
    if 'arr_group' not in df_clean.columns:
        df_clean['arr_group'] = pd.cut(
            df_clean['annual_revenue'],
            bins=[0, 5000, 15000, float('inf')],
            labels=['Low ARR (<$5k)', 'Mid ARR ($5k-$15k)', 'High ARR ($15k+)']
        )
    
    df_with_arr['arr_group'] = df_with_arr['arr_group'].astype(str)
    df_with_arr.loc[df_with_arr['is_active'] == 0, 'arr_group'] = 'Inactive (No ARR)'
    
    # Build node labels
    node_labels = []
    
    # Level 1: Portfolio Groups
    portfolio_nodes = sorted(df_clean['portfolio_group'].unique().astype(str))
    node_labels.extend(portfolio_nodes)
    
    # Level 2: Plan Types
    plan_nodes = sorted(df_clean['plan_type'].unique())
    node_labels.extend(plan_nodes)
    
    # Level 3: Event Diversity Groups
    diversity_nodes = sorted([str(x) for x in df_clean['event_diversity_group'].unique()], reverse=True)
    node_labels.extend(diversity_nodes)
    
    # Level 4: Active Status
    status_nodes = sorted([str(x) for x in df_clean['active_status'].unique()])
    node_labels.extend(status_nodes)
    
    # Level 5: ARR Groups
    arr_nodes = sorted([str(x) for x in df_with_arr['arr_group'].unique()])
    node_labels.extend(arr_nodes)
    
    # Create node index mapping
    node_dict = {str(label): idx for idx, label in enumerate(node_labels)}
    
    # Build links
    sources = []
    targets = []
    values = []
    link_labels = []
    
    # Flow 1: Portfolio → Plan
    flow1 = df_clean.groupby(['portfolio_group', 'plan_type']).agg({
        'user_id': 'count',
        'annual_revenue': 'sum'
    }).reset_index()
    
    for _, row in flow1.iterrows():
        sources.append(node_dict[str(row['portfolio_group'])])
        targets.append(node_dict[str(row['plan_type'])])
        values.append(row['user_id'])
        link_labels.append(f"{row['user_id']} users, ${row['annual_revenue']:,.0f} ARR")
    
    # Flow 2: Plan → Event Diversity
    flow2 = df_clean.groupby(['plan_type', 'event_diversity_group']).agg({
        'user_id': 'count',
        'annual_revenue': 'sum'
    }).reset_index()
    
    for _, row in flow2.iterrows():
        sources.append(node_dict[str(row['plan_type'])])
        targets.append(node_dict[str(row['event_diversity_group'])])
        values.append(row['user_id'])
        link_labels.append(f"{row['user_id']} users, ${row['annual_revenue']:,.0f} ARR")
    
    # Flow 3: Event Diversity → Active Status
    flow3 = df_clean.groupby(['event_diversity_group', 'active_status']).agg({
        'user_id': 'count',
        'annual_revenue': 'sum'
    }).reset_index()
    
    for _, row in flow3.iterrows():
        sources.append(node_dict[str(row['event_diversity_group'])])
        targets.append(node_dict[str(row['active_status'])])
        values.append(row['user_id'])
        link_labels.append(f"{row['user_id']} users, ${row['annual_revenue']:,.0f} ARR")
    
    # Flow 4: Active Status → ARR Group
    flow4 = df_with_arr.groupby(['active_status', 'arr_group']).agg({
        'user_id': 'count',
        'annual_revenue': 'sum'
    }).reset_index()
    
    for _, row in flow4.iterrows():
        sources.append(node_dict[str(row['active_status'])])
        targets.append(node_dict[str(row['arr_group'])])
        values.append(row['user_id'])
        link_labels.append(f"{row['user_id']} users, ${row['annual_revenue']:,.0f} ARR")
    
    # Create Sankey diagram
    fig = go.Figure(data=[go.Sankey(
        node=dict(
            pad=15,
            thickness=20,
            line=dict(color="black", width=0.5),
            label=node_labels,
            color=['lightblue' if 'unit' in str(n) else 
                   'lightgreen' if n in ['starter', 'pro', 'enterprise', 'premium'] else
                   'gold' if 'events' in str(n) else
                   'lightcoral' if n == 'Inactive' else 
                   'salmon' if 'Inactive' in str(n) else
                   'lightseagreen'
                   for n in node_labels]
        ),
        link=dict(
            source=sources,
            target=targets,
            value=values,
            label=link_labels
        )
    )])
    
    fig.update_layout(
        title=f"User Flow: Portfolio → Plan → Event Diversity → Active Status → ARR",
        font=dict(size=10),
        height=900
    )
    
    return fig

print("Event diversity Sankey function defined!")

Event diversity Sankey function defined!


In [ ]:
# Create the event diversity Sankey diagram
print("\n" + "="*100)
print("EVENT DIVERSITY FLOW ANALYSIS")
print("="*100)

# Create and show the Sankey
fig = create_event_diversity_sankey()
fig.show()

print("\nThis diagram shows how users flow through:")
print("  Portfolio Size → Plan Type → Event Diversity (how many features used) → Active Status → Revenue Tier")
print("\nEvent diversity groups:")
print("  • 0-4 events: Light users - minimal feature adoption")
print("  • 5-7 events: Moderate users - good feature adoption")
print("  • 7+ events: Power users - extensive feature adoption")


EVENT DIVERSITY FLOW ANALYSIS


/var/folders/dt/ygs29g2d69z6j_kp3w6648340000gn/T/ipykernel_86856/1003795851.py:55: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

/var/folders/dt/ygs29g2d69z6j_kp3w6648340000gn/T/ipykernel_86856/1003795851.py:67: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

/var/folders/dt/ygs29g2d69z6j_kp3w6648340000gn/T/ipykernel_86856/1003795851.py:79: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.




This diagram shows how users flow through:
  Portfolio Size → Plan Type → Event Diversity (how many features used) → Active Status → Revenue Tier

Event diversity groups:
  • 0-4 events: Light users - minimal feature adoption
  • 5-7 events: Moderate users - good feature adoption
  • 7+ events: Power users - extensive feature adoption


In [ ]:
# Event Diversity Impact Analysis
print("="*100)
print("EVENT DIVERSITY IMPACT: How Feature Adoption Drives Value")
print("="*100)

diversity_impact = joined_df.groupby('event_diversity_group').agg({
    'user_id': 'count',
    'annual_revenue': ['sum', 'mean'],
    'is_active': lambda x: (x.sum() / len(x) * 100),
    'nps_score': 'mean',
    'support_tickets_last_90d': 'mean'
}).round(2)

diversity_impact.columns = ['customer_count', 'total_arr', 'avg_arr', 'active_rate_%', 'avg_nps', 'avg_support_tickets']
diversity_impact['pct_of_customers'] = (diversity_impact['customer_count'] / diversity_impact['customer_count'].sum() * 100).round(1)

print("\nEvent Diversity Performance:")
display(diversity_impact.sort_index())

print("\n" + "="*100)
print("KEY INSIGHTS")
print("="*100)

# Compare power users vs light users
power_users = diversity_impact.loc['7+ events']
light_users = diversity_impact.loc['0-4 events']

arr_lift = power_users['avg_arr'] - light_users['avg_arr']
active_lift = power_users['active_rate_%'] - light_users['active_rate_%']

print(f"\nPower Users (7+ Events) vs Light Users (0-4 Events):")
print(f"  • ARR Lift: ${arr_lift:,.0f} higher per customer ({((power_users['avg_arr'] / light_users['avg_arr'] - 1) * 100):.1f}% increase)")
print(f"  • Active Rate Lift: {active_lift:.1f}% higher")
print(f"  • NPS Difference: {power_users['avg_nps'] - light_users['avg_nps']:.1f} points")

print(f"\nFeature adoption opportunity:")
moderate = diversity_impact.loc['5-7 events']
print(f"  • {moderate['customer_count']:.0f} customers ({moderate['pct_of_customers']:.1f}%) are using 5-7 features")
print(f"  • Average ARR: ${moderate['avg_arr']:,.0f}")
print(f"  • Potential to upgrade them to power users (7+) for ${arr_lift - (power_users['avg_arr'] - moderate['avg_arr']):,.0f} lift per customer")

print(f"\nLow adoption concern:")
print(f"  • {light_users['customer_count']:.0f} customers ({light_users['pct_of_customers']:.1f}%) are using 0-4 features")
print(f"  • Active rate: {light_users['active_rate_%']:.1f}% (vs {power_users['active_rate_%']:.1f}% for power users)")
print(f"  • High risk for churn - need feature adoption campaigns")

EVENT DIVERSITY IMPACT: How Feature Adoption Drives Value

Event Diversity Performance:


/var/folders/dt/ygs29g2d69z6j_kp3w6648340000gn/T/ipykernel_86856/2655057165.py:6: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



,customer_count,total_arr,avg_arr,active_rate_%,avg_nps,avg_support_tickets,pct_of_customers
event_diversity_group,,,,,,,
0-4 events,443,838420.89,1892.60,79.46,21.18,0.25,4.4
5-7 events,6391,15879105.37,2484.60,71.30,18.75,0.93,63.9
7+ events,3166,13975469.58,4414.24,67.34,16.19,1.99,31.7



KEY INSIGHTS

Power Users (7+ Events) vs Light Users (0-4 Events):
  • ARR Lift: $2,522 higher per customer (133.2% increase)
  • Active Rate Lift: -12.1% higher
  • NPS Difference: -5.0 points

Feature adoption opportunity:
  • 6391 customers (63.9%) are using 5-7 features
  • Average ARR: $2,485
  • Potential to upgrade them to power users (7+) for $592 lift per customer

Low adoption concern:
  • 443 customers (4.4%) are using 0-4 features
  • Active rate: 79.5% (vs 67.3% for power users)
  • High risk for churn - need feature adoption campaigns
